In [9]:
!pip3 install pandas matplotlib fpdf



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import os


In [11]:
# Путь к файлу
file_path = '/Users/nikitamarshchonok/Desktop/Energy_Report_Generator/data/Energy_consumption_dataset.csv'

# Загрузка данных
data = pd.read_csv(file_path)

# Вывод первых 5 строк для проверки
print(data.head())


   Month  Hour DayOfWeek Holiday  Temperature   Humidity  SquareFootage  \
0      1     0  Saturday      No    25.139433  43.431581    1565.693999   
1      1     1  Saturday      No    27.731651  54.225919    1411.064918   
2      1     2  Saturday      No    28.704277  58.907658    1755.715009   
3      1     3  Saturday      No    20.080469  50.371637    1452.316318   
4      1     4  Saturday      No    23.097359  51.401421    1094.130359   

   Occupancy HVACUsage LightingUsage  RenewableEnergy  EnergyConsumption  
0          5        On           Off         2.774699          75.364373  
1          1        On            On        21.831384          83.401855  
2          2       Off           Off         6.764672          78.270888  
3          1       Off            On         8.623447          56.519850  
4          9        On           Off         3.071969          70.811732  


In [12]:
# Основная информация
print("Информация о данных:")
print(data.info())

# Основная статистика
print("\nОписательная статистика:")
print(data.describe(include='all'))


Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Month              5000 non-null   int64  
 1   Hour               5000 non-null   int64  
 2   DayOfWeek          5000 non-null   object 
 3   Holiday            5000 non-null   object 
 4   Temperature        5000 non-null   float64
 5   Humidity           5000 non-null   float64
 6   SquareFootage      5000 non-null   float64
 7   Occupancy          5000 non-null   int64  
 8   HVACUsage          5000 non-null   object 
 9   LightingUsage      5000 non-null   object 
 10  RenewableEnergy    5000 non-null   float64
 11  EnergyConsumption  5000 non-null   float64
dtypes: float64(5), int64(3), object(4)
memory usage: 468.9+ KB
None

Описательная статистика:
              Month         Hour DayOfWeek Holiday  Temperature     Humidity  \
count   5000.000000  5000

In [20]:
# Папка для сохранения графиков
output_dir = '/Users/nikitamarshchonok/Desktop/Energy_Report_Generator/reports/PNG'
os.makedirs(output_dir, exist_ok=True)

# Гистограммы для числовых столбцов
numeric_columns = data.select_dtypes(include='number').columns
for column in numeric_columns:
    plt.figure(figsize=(8, 5))
    data[column].hist(bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f"Distribution of {column}")
    plt.xlabel(column)
    plt.ylabel("Frequency")
    plt.grid(axis='y', alpha=0.75)
    plt.savefig(f"{output_dir}/{column}_distribution.png")
    plt.close()


In [21]:
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Energy Report', border=0, ln=1, align='C')
        self.ln(10)

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, ln=1, align='L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 10)
        self.multi_cell(0, 10, body)
        self.ln()

# Создаем PDF
pdf = PDF()
pdf.add_page()

# Включаем описания
pdf.chapter_title("Dataset Overview")
pdf.chapter_body(f"Columns: {', '.join(data.columns)}")
pdf.chapter_body(f"Number of Rows: {len(data)}")
pdf.chapter_body(f"Numerical Columns: {', '.join(numeric_columns)}")

# Включаем графики
for column in numeric_columns:
    pdf.add_page()
    pdf.chapter_title(f"Visualization: {column}")
    pdf.image(f"{output_dir}/{column}_distribution.png", x=10, y=40, w=180)

# Сохраняем PDF
pdf_output = "/Users/nikitamarshchonok/Desktop/Energy_Report_Generator/reports/reportsEnergy_Report.pdf"
pdf.output(pdf_output)
print(f"PDF report saved to: {pdf_output}")


PDF report saved to: /Users/nikitamarshchonok/Desktop/Energy_Report_Generator/reports/reportsEnergy_Report.pdf
